In [1]:
!pip install requests
!pip install autopep8
!pip install  urllib
!pip install  streamlit

ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib


In [2]:
import sys
import os
import requests
import datetime
from  urllib.parse import urlencode
import pandas as pd
import altair as alt
import streamlit as st
import base64

In [3]:
#Auth dashbord https://developer.spotify.com/dashboard/applications
#Env value setting => open jupyter  https://www.twilio.com/blog/2017/01/how-to-set-environment-variables.html
# client_id = os.environ.get("CLIENTID")
# client_secret=os.environ.get("CLIENTSEC")
client_id ="7ec0b77501cf4109a5f2785a3006341d"
client_secret="af4ae97c8eac47688177a2f4b3a59a41"

In [4]:
client_creds=f"{client_id}:{client_secret}"
type(client_creds)

str

In [5]:
#VALUE: Basic <base64 encoded client_id:client_secret>"
import base64
client_creds=f"{client_id}:{client_secret}"
print(type(client_creds))
     
client_creds_b64=base64.b64encode(client_creds.encode()) 
client_creds_b64

<class 'str'>


b'N2VjMGI3NzUwMWNmNDEwOWE1ZjI3ODVhMzAwNjM0MWQ6YWY0YWU5N2M4ZWFjNDc2ODgxNzdhMmY0YjNhNTlhNDE='

In [6]:
#Request Access Token
#Flow https://developer.spotify.com/documentation/general/guides/authorization/code-flow/
token_url='https://accounts.spotify.com/api/token'
method="POST"

token_data={
    "grant_type": 'client_credentials'
}

# request must include the following HTTP headers:,Value
#header_parameter

token_headers={
    "Authorization": f"Basic {client_creds_b64.decode()}"#Basic <base64 encoded client_id:client_secret>"
}


In [7]:
r=requests.post(token_url,data=token_data,headers=token_headers)
print(r.json())
valid_request=r.status_code in range(200,299)


{'error': 'invalid_client', 'error_description': 'Invalid client'}


In [8]:
r=requests.post(token_url,data=token_data,headers=token_headers)
print(r.json())
valid_request=r.status_code in range(200,299)

if valid_request:
    token_response_data=r.json()
    now=datetime.datetime.now()
    access_token=token_response_data["access_token"]
    exprires_in=token_response_data['expires_in']
    expires=now+datetime.timedelta(seconds=exprires_in)
    didi_expires=expires < now



{'error': 'invalid_client', 'error_description': 'Invalid client'}


In [9]:
#class
class SpotifyAPI(object):
    access_token=None
    access_token_expires=datetime.datetime.now()
    access_token_did_expires=True
    client_id=None
    client_secret=None    
    token_url='https://accounts.spotify.com/api/token'
    
    def __init__(self,client_id,client_secret,*args,**kwargs):
        super().__init__(*args,**kwargs)#inherit from another class
        self.client_id=client_id
        self.client_secret=client_secret
        
    def get_client_credentials(self):
        """
        return 64 string
        """
        
        client_id=self.client_id
        client_secret=self.client_secret
        if client_secret==None or client_id== None:
            raise Exception("you must set client id and client secret")
        client_creds=f"{client_id}:{client_secret}"
        client_creds_b64=base64.b64encode(client_creds.encode()) 
        return client_creds_b64.decode()

    def get_token_headers(self):
        client_creds_b64=self.get_client_credentials()
        return {
    "Authorization": f"Basic {client_creds_b64}"#Basic <base64 encoded client_id:client_secret>"
        }
            
    def get_token_data(self):
            return{
    "grant_type": 'client_credentials'
            }
    
    def perform_auth(self):
        token_url=self.token_url
        token_data=self.get_token_data()
        token_headers=self.get_token_headers()
        
        r=requests.post(token_url,data=token_data,headers=token_headers)

        if r.status_code not in range(200,299):
                return False
        token_response_data=r.json()
        now=datetime.datetime.now()
        data=r.json()
        
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
        

In [10]:
spotify=SpotifyAPI(client_id,client_secret)

In [11]:
spotify.perform_auth()

False

In [13]:
access_token=spotify.access_token

In [129]:
headers = {
    "Authorization": f"Bearer {access_token}"
}
endpoint = "https://api.spotify.com/v1/search"
data = urlencode({"q": "kill bill", "type": "track"})
print(data)

lookup_url = f"{endpoint}?{data}"
print(lookup_url)
r = requests.get(lookup_url, headers=headers)
print(r.status_code)

q=kill+bill&type=track
https://api.spotify.com/v1/search?q=kill+bill&type=track
401


In [130]:
r.json()

{'error': {'status': 401, 'message': 'Invalid access token'}}

In [189]:
Name_of_Movie="titanic"

In [190]:
import sys
import os
import requests
import datetime
from  urllib.parse import urlencode
import base64
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
if 'client_id' and 'client_secret'in os.environ:
    client_id = os.getenv('client_id')
    client_secret = os.getenv('client_secret')
else:
    client_id = os.environ('client_id')
    client_secret = os.environ('client_secret')

#class+search method
#another search method type https://developer.spotify.com/documentation/web-api/reference/#/operations/search


class SpotifyAPI(object):
    access_token=None
    access_token_expires=datetime.datetime.now()
    access_token_did_expires=True
    client_id=None
    client_secret=None    
    token_url='https://accounts.spotify.com/api/token'
    
    def __init__(self,client_id,client_secret,*args,**kwargs):
        super().__init__(*args,**kwargs)#inherit from another class
        self.client_id=client_id
        self.client_secret=client_secret
        
    def get_client_credentials(self):
        """
        return 64 string
        """
        
        client_id=self.client_id
        client_secret=self.client_secret
        if client_secret==None or client_id== None:
            raise Exception("you must set client id and client secret")
        client_creds=f"{client_id}:{client_secret}"
        client_creds_b64=base64.b64encode(client_creds.encode()) 
        return client_creds_b64.decode()

    def get_token_headers(self):
        client_creds_b64=self.get_client_credentials()
        return {
    "Authorization": f"Basic {client_creds_b64}"#Basic <base64 encoded client_id:client_secret>"
        }
            
    def get_token_data(self):
            return{
    "grant_type": 'client_credentials'
            }
    
    def perform_auth(self):
        token_url=self.token_url
        token_data=self.get_token_data()
        token_headers=self.get_token_headers()
        
        r=requests.post(token_url,data=token_data,headers=token_headers)

        if r.status_code not in range(200,299):
            raise Exception("Not authenticate client")
        token_response_data=r.json()
        now=datetime.datetime.now()
        data=r.json()
        
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
                
        token=self.access_token
        expires=self.access_token_expires
        now=datetime.datetime.now()
        
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token==None:
            self.perform_auth()
            return self.get_access_token
            
        return token
    
    #header
    def get_resource_header(self):
            access_token = self.get_access_token()
            headers = {
                "Authorization": f"Bearer {access_token}"
            }
            return headers

    #depending on input data, get Json
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()

    #album json
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')

    #artist json 
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')

    #research Json
    def base_search(self, query_params): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()
    
    #query : dic or string  %20 is space
    def search(self,query=None, operator=None,operator_query=None ,search_type='track'):
        if query == None:
               raise Exception("A query is required")
               
        if isinstance(query,dict):
               query=" ".join([f"{key},{value}"for key,value in query.items()])#inline 
                
        if operator != None and operator_query!=None:
            operator=operator.upper()
            if operator== "or" or operator.lower=="not":
                operator=operator.upper()
                if isinstance(operator_query,str):
                    query=f"{query}{operator}{operator_query}"
               
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)
    
    
        #playlist_json_data
    def playlist_search_json_createdata(self,query="tatanic"):
        playlists_json=self.search(query=query,search_type="playlist")

        #Data
        need_playlist= []
        for i, item in enumerate(playlists_json["playlists"]["items"]):
            need_playlist.append((i,
                          item["name"], 
                          item["external_urls"]["spotify"],
                          item["id"],
                          item["images"][0]["url"],
                         
         ))
            playlist_df = pd.DataFrame(need_playlist,index=None,columns=('item','Name','PlaylistURL','ID','ImageURL'))
        return playlist_df
    
    
    def spotify_get_organised_data(self,Name_of_Movie):
            #Data
        Data = self.search({"album": f"{Name_of_Movie}"}, search_type="track")
        
        #need contents 
        need = []
        for i, item in enumerate(Data['tracks']['items']):
            track = item['album']
            track_id = item['id']
            song_name = item['name']
            popularity = item['popularity']
            need.append((i, track['artists'][0]['name'], track['name'], track_id, song_name, track['release_date'], popularity,item['album']["id"]))

        #data frame top10
        Track_df = pd.DataFrame(need, index=None, columns=('Item', 'Artist', 'Album Name', 'Id', 'Song Name', 'Release Date', 'Popularity',"albumID"))

        access_token = self.access_token
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        endpoint = "https://api.spotify.com/v1/audio-features/"

        #Full data
        Feat_df = pd.DataFrame()
        for id in Track_df['Id'].iteritems():
            track_id = id[1]
            lookup_url = f"{endpoint}{track_id}"
            ra = requests.get(lookup_url, headers=headers)
            audio_feat = ra.json()
            Features_df = pd.DataFrame(audio_feat, index=[0])
            Feat_df = Feat_df.append(Features_df)
        Full_Data = Track_df.merge(Feat_df, left_on="Id", right_on="id")

        #sort
        Sort_DF = Full_Data.sort_values(by=['Popularity'], ascending=False).head(10)

        #chart df 
        chart_df = Sort_DF[['Artist', 'Album Name', 'Song Name', 'Release Date', 'Popularity',"energy","albumID"]]
        Name_of_Feat="energy"

        return chart_df
    
 #playlist_json_data
    def playlist_search_json_createdata(self,query=None):
        query="titanic" if query is None else query
        
        playlists_json=self.search(query=query,search_type="playlist")
        
        #Data
        need_playlist= []
        for i, item in enumerate(playlists_json["playlists"]["items"]):
            need_playlist.append((i,
                          item["name"], 
                          item["external_urls"]["spotify"],
                          item["id"],
                          item["images"][0]["url"],
                         
         ))
            playlist_df = pd.DataFrame(need_playlist,index=None,columns=('item','Name','PlaylistURL','ID','ImageURL'))
        return playlist_df
    
    def get_track_from_playlist(PlaylistURL):
        client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
        sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)
        track_uri_lis= []
        
        for track in sp.playlist_tracks(playlist_URI)["items"]:
            #URI
            track_uri = track["track"]["uri"]
            
            #id
            track_id = track["track"]["id"]
            
            #Track name
            track_name = track["track"]["name"]
            popularity=track["track"]["popularity"]
            
            #Main Artist
            artist_uri = track["track"]["artists"][0]["uri"]
            artist_info = sp.artist(artist_uri)
            
            #Name, popularity, genre
            artist_name = track["track"]["artists"][0]["name"]
            artist_pop = artist_info["popularity"]
            artist_genres = artist_info["genres"]
            
            #Album
            album = track["track"]["album"]["name"]
            
            #Popularity of the track
            track_pop = track["track"]["popularity"]
            #energy
            energy=sp.audio_features(track_uri)[0]["energy"]
            
            all_lis=[album,track_name,popularity,energy,track_uri,track_id]
            track_uri_lis.append((all_lis))
            track_uri_lis

        playlist_df = pd.DataFrame(track_uri_lis,index=None,columns=('album','track_name',"popularity","energy",'track_uri',"id"))
        Track_df=playlist_df
        Sort_DF =Track_df.sort_values(by=['popularity'], ascending=False)
        Sort_DF
        
        return  Sort_DF
            
        

In [191]:
spotify=SpotifyAPI(client_id,client_secret)


In [201]:
playlist_URI=spotify.playlist_search_json_createdata()["PlaylistURL"][0]
playlist_URI

q=titanic&type=playlist


'https://open.spotify.com/playlist/1C7xHrNHW2rjRJJkNlRKuT'

In [202]:
spotify.playlist_search_json_createdata()["PlaylistURL"]

q=titanic&type=playlist


0     https://open.spotify.com/playlist/1C7xHrNHW2rj...
1     https://open.spotify.com/playlist/5AQJpvFYJa2z...
2     https://open.spotify.com/playlist/7sw39BbfYmh2...
3     https://open.spotify.com/playlist/59yb92HzA56R...
4     https://open.spotify.com/playlist/0VKg40AFSXEa...
5     https://open.spotify.com/playlist/7pcMLozZ5YHc...
6     https://open.spotify.com/playlist/4ucSam3agPZG...
7     https://open.spotify.com/playlist/5hsHRnJwQ31U...
8     https://open.spotify.com/playlist/2ykurNREIHa3...
9     https://open.spotify.com/playlist/2zu7vrQego8j...
10    https://open.spotify.com/playlist/3eVuzeNOwTJ4...
11    https://open.spotify.com/playlist/04vjEoBPc0oH...
12    https://open.spotify.com/playlist/6sswRC7bEaj3...
13    https://open.spotify.com/playlist/7qb4RyZ8nAjq...
14    https://open.spotify.com/playlist/5lWj3j7cwPEj...
15    https://open.spotify.com/playlist/7hUsvGzCJXAc...
16    https://open.spotify.com/playlist/2VNqViCMHD65...
17    https://open.spotify.com/playlist/7HVpZsgx

In [204]:
st.header(f"Song Popularity+Energy Chart")

Name_of_Feat="energy"
track_df=SpotifyAPI.get_track_from_playlist(playlist_URI)
track_df

# c = alt.Chart(track_df).mark_circle().encode(
#     alt.X('popularity', scale=alt.Scale(zero=False)), y=f'{Name_of_Feat}', color=alt.Color('popularity', scale=alt.Scale(type='log',scheme='rainbow')), 
#     size=alt.value(300), tooltip=['popularity', f'{Name_of_Feat}',"energy"]).interactive()
# st.altair_chart(c, use_container_width=False)

,album,track_name,popularity,energy,track_uri,id
0,Titanic: Original Motion Picture Soundtrack - ...,Never an Absolution,50,0.02990,spotify:track:1YaJrC7OyN9S7OJRIUhDaU,1YaJrC7OyN9S7OJRIUhDaU
3,Titanic: Music from the Motion Picture Soundtrack,Rose,48,0.03150,spotify:track:5SPGvCYQNZtSz2dRor0fls,5SPGvCYQNZtSz2dRor0fls
13,Titanic: Music from the Motion Picture Soundtrack,"My Heart Will Go On - Love Theme from ""Titanic""",47,0.10300,spotify:track:42lyCSXzWizBfGmqE1Silk,42lyCSXzWizBfGmqE1Silk
2,Titanic: Original Motion Picture Soundtrack - ...,Southampton,46,0.11100,spotify:track:6nqfi9YaqwFsLzPI9JACob,6nqfi9YaqwFsLzPI9JACob
14,Titanic: Music from the Motion Picture Soundtrack,Hymn to the Sea,45,0.07090,spotify:track:77vAuUDLRCp4pa5FBfx9F6,77vAuUDLRCp4pa5FBfx9F6
1,Titanic: Original Motion Picture Soundtrack - ...,Distant Memories,43,0.04200,spotify:track:7gtrC6cqQ2m232H5R3bpIU,7gtrC6cqQ2m232H5R3bpIU
11,Titanic: Music from the Motion Picture Soundtrack,A Life So Changed,40,0.01240,spotify:track:5Ex7XDkNhcW67bHC8Ip1OT,5Ex7XDkNhcW67bHC8Ip1OT
6,Titanic: Music from the Motion Picture Soundtrack,"""Hard to Starboard""",39,0.12000,spotify:track:7FdV1Uaqjqwfw2IXcnnM77,7FdV1Uaqjqwfw2IXcnnM77
7,Titanic: Music from the Motion Picture Soundtrack,"Unable to Stay, Unwilling to Leave",39,0.14400,spotify:track:6Rlg96GUPul3qfaU2NEYtv,6Rlg96GUPul3qfaU2NEYtv
4,Titanic: Music from the Motion Picture Soundtrack,Leaving Port,38,0.13000,spotify:track:0uytU4FSLbiJjtplmEMA3e,0uytU4FSLbiJjtplmEMA3e


In [183]:
spotify.spotify_get_organised_data("titanic")["albumID"]

q=album%2Ctitanic&type=track


/var/folders/tg/p4s0x3b55936405b1y1vws6h0000gn/T/ipykernel_31140/773418225.py:198: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Feat_df = Feat_df.append(Features_df)
/var/folders/tg/p4s0x3b55936405b1y1vws6h0000gn/T/ipykernel_31140/773418225.py:198: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Feat_df = Feat_df.append(Features_df)
/var/folders/tg/p4s0x3b55936405b1y1vws6h0000gn/T/ipykernel_31140/773418225.py:198: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Feat_df = Feat_df.append(Features_df)


1    3XwgFTxiWxep4s1VjPEaHs
0    24YoVjBTadmRW2Zhc9ou9S
2    2mgkd00677MkIIw09g0kss
Name: albumID, dtype: object

In [135]:
#playlist json 
name="kill bill"
playlists_json=spotify.search(query=name,search_type="playlist")
playlists_json

q=kill+bill&type=playlist


{'playlists': {'href': 'https://api.spotify.com/v1/search?query=kill+bill&type=playlist&offset=0&limit=20',
  'items': [{'collaborative': False,
    'description': '',
    'external_urls': {'spotify': 'https://open.spotify.com/playlist/6VUYYbBUbRDyWMszBZZ8yP'},
    'href': 'https://api.spotify.com/v1/playlists/6VUYYbBUbRDyWMszBZZ8yP',
    'id': '6VUYYbBUbRDyWMszBZZ8yP',
    'images': [{'height': None,
      'url': 'https://i.scdn.co/image/ab67706c0000bebb827f8fd00fb71e5b0e06ca93',
      'width': None}],
    'name': 'Kill Bill: Vol. 1 & 2 (Original Soundtrack)',
    'owner': {'display_name': 'Alexandre Dupé',
     'external_urls': {'spotify': 'https://open.spotify.com/user/11136889703'},
     'href': 'https://api.spotify.com/v1/users/11136889703',
     'id': '11136889703',
     'type': 'user',
     'uri': 'spotify:user:11136889703'},
    'primary_color': None,
    'public': None,
    'snapshot_id': 'NjIsZjE0MDIwMzUzMTc5NGU5YzBkNjZkZWQwNzJmN2NlZjRiZjA4MzQ2Ng==',
    'tracks': {'href': 'h

In [61]:
playlist_df=spotify.playlist_search_json_createdata(query=name)
playlist_df

q=kill+bill&type=playlist


,item,Name,PlaylistURL,ID,ImageURL
0,0,Kill Bill: Vol. 1 & 2 (Original Soundtrack),https://open.spotify.com/playlist/6VUYYbBUbRDy...,6VUYYbBUbRDyWMszBZZ8yP,https://i.scdn.co/image/ab67706c0000bebb827f8f...
1,1,KILL BILL AND RAV,https://open.spotify.com/playlist/0Emv68wA7olG...,0Emv68wA7olGuMETpAUE1U,https://i.scdn.co/image/ab67706c0000bebb356068...
2,2,Kill Bill whistle,https://open.spotify.com/playlist/20yPv6JoiDed...,20yPv6JoiDed079C5cHTiP,https://i.scdn.co/image/ab67616d0000b273fecfac...
3,3,Kill Bill Vibes,https://open.spotify.com/playlist/6mlt3X9QZXra...,6mlt3X9QZXra84AoUB062x,https://i.scdn.co/image/ab67706c0000bebb14a0b3...
4,4,Kill Bill Vols. 1 & 2 | Movie Soundtracks,https://open.spotify.com/playlist/2k2Z29MFKoF0...,2k2Z29MFKoF0J80VSjwNcH,https://i.scdn.co/image/ab67616d0000b273fecfac...
5,5,This is Kill Bill: The Rapper,https://open.spotify.com/playlist/3Qfle5gaQ9Zs...,3Qfle5gaQ9ZsX0pj8ufpIr,https://i.scdn.co/image/ab67706c0000bebbdf7ba5...
6,6,POV: you're the final girl 🔪,https://open.spotify.com/playlist/5dqlL4K7D2vW...,5dqlL4K7D2vWWgICZAIxOu,https://i.scdn.co/image/ab67706c0000bebb992112...
7,7,Kill Bill: The Rapper - Stream Safe Music,https://open.spotify.com/playlist/4NuXMDh7Haa3...,4NuXMDh7Haa3AGXppi7BC4,https://mosaic.scdn.co/640/ab67616d0000b2733bc...
8,8,serial killer playlist,https://open.spotify.com/playlist/0nJGhJfvDEXF...,0nJGhJfvDEXFnviKtgowH5,https://i.scdn.co/image/ab67706c0000bebb446544...
9,9,KILL BILL (FAST),https://open.spotify.com/playlist/1g0Mh5J1gWyk...,1g0Mh5J1gWykT9PF5YJy3P,https://mosaic.scdn.co/640/ab67616d0000b273690...


In [62]:
#top URL
playlist_URI=playlist_df["PlaylistURL"][0]
playlist_URI

'https://open.spotify.com/playlist/6VUYYbBUbRDyWMszBZZ8yP'

In [63]:
#spotipy recommendation
!pip install  spotipy

In [64]:
import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)
sp

In [65]:
track_uris=[x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]
track_uris

['spotify:track:3Vr406z58yyyO2Ofh1fbqA',
 'spotify:track:1s2I9Q7zAE78m7aVZOq3ug',
 'spotify:track:1aR6KEUmuCvrI5Q7LhmQLN',
 'spotify:track:3Y9wdXMh6sNt9fUwUqTqb7',
 'spotify:track:0ViM3sjKdaCQR488Dxw62D',
 'spotify:track:3hMir99e3Uk4MfH4n7LQX1',
 'spotify:track:4f4Lmrpw0Me9VOaueFidlB',
 'spotify:track:1hJkuyZmeXesANjQuHvbtH',
 'spotify:track:33lSCA6SI87sJYPZFtSxbc',
 'spotify:track:173FQi5O8avUWQt2UFk7Zd',
 'spotify:track:2UYI1b74b40ZnUoxebfzfx',
 'spotify:track:3OQEVjVEQ6CvfAJ6XhTs6c',
 'spotify:track:07prU8izeUwN1xcTuMXvg3',
 'spotify:track:0Rynk2V7LyLgBUjTMxvbEJ',
 'spotify:track:6Q32Vkucx2qeuVyBd3NiFZ',
 'spotify:track:66eoy2pfLjmLCiQrWH7pcq',
 'spotify:track:6GA8fbPKP6KZ3fP75AIjRI',
 'spotify:track:1XzeUXHiKRf6YWY8O4FXBB',
 'spotify:track:7EbPUFnL7ujRxxxElUWZvg',
 'spotify:track:1RHXMK6iFO2Qf7ph3a2aTy',
 'spotify:track:4QkvD53COA5qLA7rQ1Pa7D',
 'spotify:track:15BsONOampPaG9qWGftyhB',
 'spotify:track:6rhVKeUhQTGLIRmQOfc9Wo',
 'spotify:track:0RQOZ6q9OTvfQX8HCGzmIB',
 'spotify:track:

In [104]:
#track features
sp.audio_features(track_uri)[0]["energy"]

0.468

In [95]:
for track in sp.playlist_tracks(playlist_URI)["items"]:
    print(track["track"]["popularity"])

51
1
49
57
39
38
37
47
38
44
42
57
1
46
55
35
45
45
41
36
58
1
62
50
57
39
43
36
43
34
34
46
37
34
0
45
37
52
34
51
43


In [107]:
track_uri_lis= []
for track in sp.playlist_tracks(playlist_URI)["items"]:
    #URI
    track_uri = track["track"]["uri"]
    
    #id
    track_id = track["track"]["id"]
    
    #Track name
    track_name = track["track"]["name"]
    popularity=track["track"]["popularity"]
    
    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    
    #Name, popularity, genre
    artist_name = track["track"]["artists"][0]["name"]
    artist_pop = artist_info["popularity"]
    artist_genres = artist_info["genres"]
    
    #Album
    album = track["track"]["album"]["name"]
    
    #Popularity of the track
    track_pop = track["track"]["popularity"]
    #energy
    energy=sp.audio_features(track_uri)[0]["energy"]
    
    all_lis=[track_name,popularity,energy,track_uri,track_id,album]
    track_uri_lis.append((all_lis))
track_uri_lis

playlist_df = pd.DataFrame(track_uri_lis,index=None,columns=('track_name',"popularity","energy",'track_uri',"id",'album'))
Track_df=playlist_df
    
#     need_playlist= []
#     for i, item in enumerate(playlists_json["playlists"]["items"]):
#         need_playlist.append((i,
#                               track_uri, 
#                           track_name,
#                           track_pop,
#                          ))
#     need_playlist
# #     playlist_df = pd.DataFrame(need_playlist,index=None,columns=('item','Name','PlaylistURL','ID','ImageURL'))
# #     playlist_df


In [108]:
Track_df

,track_name,popularity,energy,track_uri,id,album
0,Funky Fanfare,51,0.8310,spotify:track:3Vr406z58yyyO2Ofh1fbqA,3Vr406z58yyyO2Ofh1fbqA,The Kpm 1000 Series - Volume 1
1,Bang Bang - My Baby Shot Me Down,1,0.1200,spotify:track:1s2I9Q7zAE78m7aVZOq3ug,1s2I9Q7zAE78m7aVZOq3ug,Kill Bill Vol. 1 Original Soundtrack (PA Version)
2,That Certain Female,49,0.6260,spotify:track:1aR6KEUmuCvrI5Q7LhmQLN,1aR6KEUmuCvrI5Q7LhmQLN,Kill Bill Vol. 1 Original Soundtrack (PA Version)
3,Twisted Nerve,57,0.1660,spotify:track:3Y9wdXMh6sNt9fUwUqTqb7,3Y9wdXMh6sNt9fUwUqTqb7,Kill Bill Vol. 1 Original Soundtrack (PA Version)
4,Sette note - Pt. 1 - Single Version,39,0.3940,spotify:track:0ViM3sjKdaCQR488Dxw62D,0ViM3sjKdaCQR488Dxw62D,Sette note in nero
5,"Main Title ""Truck Turner""",38,0.7650,spotify:track:3hMir99e3Uk4MfH4n7LQX1,3hMir99e3Uk4MfH4n7LQX1,Truck Turner (Original Motion Picture Soundtrack)
6,Storm Rider - The Grand Duel: Parte Sesta,37,0.0421,spotify:track:4f4Lmrpw0Me9VOaueFidlB,4f4Lmrpw0Me9VOaueFidlB,The Grand Duel - Storm Rider (Original Motion ...
7,The Grand Duel - Parte Prima,47,0.2490,spotify:track:1hJkuyZmeXesANjQuHvbtH,1hJkuyZmeXesANjQuHvbtH,Kill Bill Vol. 1 Original Soundtrack (PA Version)
8,"A Long Day of Vengeance (From ""Kill Bill Vol. 1"")",38,0.3660,spotify:track:33lSCA6SI87sJYPZFtSxbc,33lSCA6SI87sJYPZFtSxbc,Quentin Tarantino Music Collection
9,Run Fay Run,44,0.9080,spotify:track:173FQi5O8avUWQt2UFk7Zd,173FQi5O8avUWQt2UFk7Zd,Kill Bill Vol. 1 Original Soundtrack (PA Version)


In [109]:
Sort_DF =Track_df.sort_values(by=['popularity'], ascending=False)
Sort_DF

#chart df 
# chart_df = Sort_DF[['Artist', 'Album Name', 'Song Name', 'Release Date', 'Popularity',"energy","albumID"]]
# Name_of_Feat="energy"

# return chart_df

,track_name,popularity,energy,track_uri,id,album
22,Don't Let Me Be Misunderstood,62,0.8150,spotify:track:6rhVKeUhQTGLIRmQOfc9Wo,6rhVKeUhQTGLIRmQOfc9Wo,Kill Bill Vol. 1 Original Soundtrack (PA Version)
20,Nobody But Me,58,0.7500,spotify:track:4QkvD53COA5qLA7rQ1Pa7D,4QkvD53COA5qLA7rQ1Pa7D,Nobody But Me
11,The Lonely Shepherd,57,0.3040,spotify:track:3OQEVjVEQ6CvfAJ6XhTs6c,3OQEVjVEQ6CvfAJ6XhTs6c,Kill Bill Vol. 1 Original Soundtrack (PA Version)
3,Twisted Nerve,57,0.1660,spotify:track:3Y9wdXMh6sNt9fUwUqTqb7,3Y9wdXMh6sNt9fUwUqTqb7,Kill Bill Vol. 1 Original Soundtrack (PA Version)
24,Goodnight Moon,57,0.6740,spotify:track:5jQPDVjgSGnNYl8G3H3qul,5jQPDVjgSGnNYl8G3H3qul,Kill Bill Vol. 2 Original Soundtrack
14,Battle Without Honor Or Humanity,55,0.8440,spotify:track:6Q32Vkucx2qeuVyBd3NiFZ,6Q32Vkucx2qeuVyBd3NiFZ,Kill Bill Vol. 1 Original Soundtrack (PA Version)
37,About Her,52,0.6000,spotify:track:0WN1ICX0u0NenOccX8gRp3,0WN1ICX0u0NenOccX8gRp3,Kill Bill Vol. 2 Original Soundtrack
39,Malaguena Salerosa,51,0.7770,spotify:track:4oIOXVdUUHaxLyeosHWABU,4oIOXVdUUHaxLyeosHWABU,Kill Bill Vol. 2 Original Soundtrack
0,Funky Fanfare,51,0.8310,spotify:track:3Vr406z58yyyO2Ofh1fbqA,3Vr406z58yyyO2Ofh1fbqA,The Kpm 1000 Series - Volume 1
23,The Flower Of Carnage,50,0.3970,spotify:track:0RQOZ6q9OTvfQX8HCGzmIB,0RQOZ6q9OTvfQX8HCGzmIB,Kill Bill Vol. 1 Original Soundtrack (PA Version)
